In [30]:
import numpy as np
from numpy import load
import random
from sklearn.linear_model import Ridge
from fancyimpute import SoftImpute, BiScaler
from sklearn.linear_model import LinearRegression as reg
from sklearn.linear_model import ElasticNetCV
import itertools
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import warnings
import os, sys
warnings.filterwarnings('ignore')

data=np.load('august2020-exercise1.npz')

#print(data.files) # view the files in the npz

#function to normalize the data
def normalize(matrix):
    normed=(matrix-matrix.mean(axis=0))/matrix.std(axis=0)
    return normed

######## train data ########
X_lr_small_train=normalize(data['X_lr_small_train'])
X_lr_big_train=normalize(data['X_lr_big_train'])
X_hr_small_train=normalize(data['X_hr_small_train'])
X_hr_big_train=normalize(data['X_hr_big_train'])

y_lr_small_train=normalize(data['y_lr_small_train'])
y_lr_big_train=normalize(data['y_lr_big_train'])
y_hr_small_train=normalize(data['y_hr_small_train'])
y_hr_big_train=normalize(data['y_hr_big_train'])
######## test data#########
X_lr_test=normalize(data['X_lr_test'])
X_hr_test=normalize(data['X_hr_test'])

y_lr_test=normalize(data['y_lr_test'])
y_hr_test=normalize(data['y_hr_test'])

def compRandomFunc(array,probability): # probability=0.7
    n=array.shape[0]
    p=array.shape[1]
    array=array.astype("float")
    for i in range(n):    
        for j in range(p):
            r=random.uniform(0, 1)
            if r < probability:
                array[i,j]=np.NaN
    return array

def missingByFeature(array,p1,p2): #p1=0.875, p2=0.8
    n=array.shape[0]
    p=array.shape[1]
    array=array.astype("float")
    featureList=np.array([])
    for i in range(p):
        r = random.uniform(0, 1)
        if r<p1:
            featureList=np.append(featureList,i)
    for i in featureList:  # i is the counter for feature not sample size
        i=int(i)
        for j in range(n):
            r2=random.uniform(0,1)
            if r2<p2:
                array[j,i]=np.NaN
    return array

def meanImputation(feature,data):
    imputed=np.nanmean(data[:,feature])
    return imputed

# creating data with missing values
import math
probability=0.7
p1=0.875
p2=0.8
# return the imputed matrix with mean imputation
def imputetMissing(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if math.isnan(matrix[i,j]) is True:
                matrix[i,j]=meanImputation(j,matrix)
    return matrix

def test(models, X_train,y_train,X_test,y_test, iterations = 100):
    results = {}
    for i in models:
        r2_train = []
        r2_test = []
        for j in range(iterations):
            r2_test.append(metrics.r2_score(y_test,
                                            models[i].fit(X_train, 
                                                         y_train).predict(X_test)))

            r2_train.append(metrics.r2_score(y_train, 
                                             models[i].fit(X_train, 
                                                          y_train).predict(X_train)))
        results[i] = [np.mean(r2_train),np.mean(r2_test)]
    return pd.DataFrame(results)

def importantFeatures(best_model,X_train,y_train,BS_size):
    importantFeatures=np.array([])
    
    for i in range(100):
        #creatig bootstrap boots
        x_bag=np.empty((0,X_train.shape[1]))
        y_bag=np.array([])
        for j in range(BS_size): 
            randIndex=np.random.randint(len(X_train), size=1)
            x_sample=X_train[randIndex,:]
            y_sample=y_train[randIndex]
                
            x_bag=np.concatenate((x_bag,x_sample))
            y_bag=np.append(y_bag,y_sample)             
        # test the estimator on bootstrap sample 100 times
        #skippa grid search och kör dirr på bästa hyperparametrarna för att spara tid
        bestEstimator=best_model.fit(x_bag,y_bag)
        
        rfeBest=RFE(bestEstimator)
        X_rfe=rfeBest.fit_transform(x_bag,y_bag)
        bestEstimator.fit(X_rfe,y_bag)
        
        importantFeatures=np.append(importantFeatures,np.where(rfeBest.ranking_==1))
        
    return importantFeatures

def featureImportance(featureList):
    unique_elements, counts_elements = np.unique(featureList, return_counts=True)
    return unique_elements, counts_elements

def mostCommonFeatures(occurrence):
    mostCommonIndex=np.array([])
    for i in range(len(occurrence)):
        if occurrence[i]>=80:
            mostCommonIndex=np.append(mostCommonIndex,[i])
    mostCommonIndex=np.array(mostCommonIndex,dtype=np.int8)
    mostCommonFeatures=featureIndex[mostCommonIndex]
    return mostCommonFeatures

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout




In [21]:
models = {'OLS': linear_model.LinearRegression(),
         'Lasso': linear_model.Lasso(),
         'Ridge': linear_model.Ridge(),}

lasso_params = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03]}
ridge_params = {'alpha':[200, 250, 300, 400, 500]}
eNet_params={'alpha': [0, 0.5, 0.1, 0.01, 0.001],
             'l1_ratio': [0, 0.25, 0.5, 0.75, 1]}

models2 = {'OLS': linear_model.LinearRegression(),
           'Lasso': GridSearchCV(linear_model.Lasso(), 
                               param_grid=lasso_params).fit(X_lr_big_train,y_lr_big_train).best_estimator_,
           'Ridge': GridSearchCV(linear_model.Ridge(), 
                               param_grid=ridge_params).fit(X_lr_big_train,y_lr_big_train).best_estimator_,
          'Elastic net':GridSearchCV(linear_model.ElasticNet(), 
                               param_grid=eNet_params).fit(X_lr_big_train,y_lr_big_train).best_estimator_}

print('low rate big data')
test(models2, X_lr_big_train,y_lr_big_train,X_lr_test,y_lr_test)

print(GridSearchCV(linear_model.ElasticNet(),param_grid=eNet_params).fit(X_lr_big_train,y_lr_big_train).best_estimator_)

low rate big data
ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)


In [22]:
bestModel=ElasticNet(l1_ratio=0,alpha=0.1)
bestModel.fit(X_lr_big_train,y_lr_big_train)
print('Best model score:',bestModel.score(X_lr_test,y_lr_test))
#print('Best model coefficients:',bestModel.coef_)

Best model score: 0.3187339776698195


In [35]:
BS_size_big=1000
feature_lr_big=importantFeatures(bestModel,X_lr_big_train,y_lr_big_train,BS_size_big)

In [37]:
mostCommonFeatures(occurrence)

array([ 13.,  21.,  36.,  63.,  73.,  81.,  87.,  98., 113.,  73.,  81.,
        88., 104., 111., 115., 116., 143.])

In [36]:
featureIndex,occurrence=featureImportance(feature_lr_big)
len(mostCommonFeatures(occurrence))

17

In [31]:
#random missing and softImpute
softImpute = SoftImpute()
biscaler = BiScaler()
softImputeScore=np.array([])
scale=np.array([0.1,0.25,0.5,0.75])

for i in range(10):
    with HiddenPrints():
        X_randomMissingSoft=compRandomFunc(X_lr_big_train,probability) #here is the line where you enable missing data
        X_randomMissingSoft_normalized = biscaler.fit_transform(X_randomMissingSoft)
        
        lambda_0=np.nanmax(X_randomMissingSoft_normalized)
        bestOfLambdaScore=np.array([])
        for j in range(len(scale)):
            softImpute=SoftImpute(shrinkage_value=lambda_0*scale[j])
            
            X_imputed_soft_normalized = softImpute.fit_transform(X_randomMissingSoft_normalized)
            X_imputed_soft = biscaler.inverse_transform(X_imputed_soft_normalized)
            
            bestModel.fit(X_imputed_soft,y_lr_big_train)
            bestOfLambdaScore=np.append(bestOfLambdaScore,bestModel.score(X_lr_test,y_lr_test))
            
        bestLambdaIndex=np.where(bestOfLambdaScore==np.max(bestOfLambdaScore))
        softImpute=SoftImpute(shrinkage_value=lambda_0*scale[bestLambdaIndex])
        X_imputed_soft_normalized = softImpute.fit_transform(X_randomMissingSoft_normalized)
        X_imputed_soft = biscaler.inverse_transform(X_imputed_soft_normalized)
        
        softImputeScore=np.append(softImputeScore,np.max(bestOfLambdaScore))
        
    #do the feature selection thing
    softImpute_feature=importantFeatures(bestModel,X_imputed_soft,y_lr_big_train,BS_size_big)
    softImputed_featureIndex,softImputed_occurrence=featureImportance(softImpute_feature)
    print('There are',len(mostCommonFeatures(softImputed_occurrence)),'important features',flush=True)
    print(mostCommonFeatures(softImputed_occurrence),flush=True)
print(softImputeScore,flush=True)

There are 17 important features
[ 54.  89. 101. 108. 112. 115. 119. 131. 151. 153. 155.  40.  52.  67.
  81.  90. 112.]
There are 21 important features
[ 26.  54.  87.  89. 101. 112. 113. 115. 131. 151. 168.  62.  67.  73.
  81.  90.  96.  97. 112. 113. 122.]
There are 15 important features
[ 89. 101. 112. 115. 124. 151. 155. 167.  52.  67.  73.  81.  90.  97.
 113.]
There are 18 important features
[ 26.  54.  87.  89. 101. 112. 115. 119. 124. 161.  40.  52.  62.  81.
  96.  97. 122. 131.]
There are 15 important features
[ 89. 101. 112. 113. 119. 124. 151.  62.  67.  79.  81.  90.  97.  99.
 113.]
There are 18 important features
[ 26.  52.  57.  87.  89. 101. 112. 115. 132. 151. 155.  52.  67.  81.
  90.  97. 104. 112.]
There are 20 important features
[ 26.  57.  87. 101. 112. 115. 119. 132. 151. 153.  40.  52.  62.  67.
  69.  81.  90.  97. 122. 131.]
There are 20 important features
[ 54.  89. 101. 112. 119. 124. 133. 136. 151. 155. 161.  40.  52.  62.
  67.  73.  81.  90.  97. 113.]


In [32]:
#softImpute with missing by feature
softImputeMBFScore=np.array([])
for i in range(10):
    with HiddenPrints():
        X_MBF_soft=missingByFeature(X_lr_big_train,p1,p2)
        X_MBF_soft_normalized=biscaler.fit_transform(X_MBF_soft)
        
        lambda_0=np.nanmax(X_randomMissingSoft_normalized)
        bestOfLambdaScore=np.array([])
        for j in range(len(scale)):
            softImpute=SoftImpute(shrinkage_value=lambda_0*scale[j])
            
            X_MBF_imputed_soft_normalized=softImpute.fit_transform(X_MBF_soft_normalized)
            X_MBF_imputed_soft=biscaler.inverse_transform(X_MBF_imputed_soft_normalized)
            
            bestModel.fit(X_MBF_imputed_soft,y_lr_big_train)
            bestOfLambdaScore=np.append(bestOfLambdaScore,bestModel.score(X_lr_test,y_lr_test))
        
        bestLambdaIndex=np.where(bestOfLambdaScore==np.max(bestOfLambdaScore))
        softImpute=SoftImpute(shrinkage_value=lambda_0*scale[bestLambdaIndex])
        
        X_MBF_imputed_soft_normalized=softImpute.fit_transform(X_MBF_soft_normalized)
        X_MBF_imputed_soft=biscaler.inverse_transform(X_MBF_imputed_soft_normalized)
        
        softImputeMBFScore=np.append(softImputeMBFScore,np.max(bestOfLambdaScore))
    
    #do the feature selection thing
    softImpute_MBF_feature=importantFeatures(bestModel,X_MBF_imputed_soft,y_lr_big_train,BS_size_big)
    softImputed_MBF_featureIndex,softImputed_MBF_occurrence=featureImportance(softImpute_MBF_feature)
    
    print('There are',len(mostCommonFeatures(softImputed_MBF_occurrence)),'important features')
    print(mostCommonFeatures(softImputed_MBF_occurrence))
print(softImputeMBFScore)

There are 13 important features
[ 26.  54.  87. 112. 115. 153. 167.  40.  62.  67.  79.  90. 104.]
There are 10 important features
[ 54. 113. 119. 124.  47.  62.  67.  79.  81.  90.]
There are 12 important features
[ 54.  57.  89. 101. 112. 115. 155. 169.  62.  96. 113. 122.]
There are 13 important features
[101. 112. 115. 119. 124. 131. 151. 153.  40.  67.  73.  81. 112.]
There are 12 important features
[ 34. 101. 131. 155.  40.  54.  62.  67.  81.  96.  97. 131.]
There are 11 important features
[ 54. 113. 124. 136. 151.  52.  62.  67.  81.  86.  90.]
There are 15 important features
[ 54.  57.  89.  98. 101. 112. 119. 132. 151.  52.  81.  90.  97. 113.
 131.]
There are 14 important features
[ 26.  52.  54.  89. 101. 119. 124. 148.  79.  81.  90.  97. 112. 122.]
There are 12 important features
[ 52.  54.  57. 101. 108. 112. 119. 151.  62.  67.  96.  97.]
There are 9 important features
[ 26.  87. 119. 155.  54.  67.  97. 113. 129.]
[0.31185328 0.3124782  0.31131603 0.3152979  0.30892148

In [33]:
# random missing and mean impute
imputeScore=np.array([])
for i in range(10): 
    
    missing_random=compRandomFunc(X_lr_big_train,probability)
    imputed_random=imputetMissing(missing_random)
    
    #doing the predective performance thing
    bestModel.fit(imputed_random,y_lr_big_train)
    imputeScore=np.append(imputeScore,bestModel.score(X_lr_test,y_lr_test))
    
    #do the feature selection thing
    imputed_feature=importantFeatures(bestModel,imputed_random,y_lr_big_train,BS_size_big)
    imputed_featureIndex,imputed_occurrence=featureImportance(imputed_feature)
    print('There are',len(mostCommonFeatures(imputed_occurrence)),'important features')
    print(mostCommonFeatures(imputed_occurrence))
    
print(imputeScore)

There are 12 important features
[ 24.  59. 101. 136.  54.  57.  62.  67.  90.  97. 100. 131.]
There are 10 important features
[ 47.  52. 124. 126. 136.  40.  67.  90. 113. 122.]
There are 6 important features
[111. 112.  40.  62.  67. 113.]
There are 11 important features
[ 57.  91. 112. 119. 129. 136.  40.  57.  62.  90. 113.]
There are 12 important features
[ 26.  30.  52.  54.  57.  59.  89. 151.  67.  81.  97. 131.]
There are 6 important features
[126. 136. 143. 155.  40. 112.]
There are 9 important features
[ 72. 101. 136.  52.  54.  67.  90.  97. 112.]
There are 11 important features
[ 26.  52.  91. 115. 124. 151.  54.  81.  86. 114. 129.]
There are 11 important features
[ 40.  72.  91. 124. 161.  47.  65.  67.  81.  90. 131.]
There are 10 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
[-0.92455693 -0.94097812 -0.98028143 -0.93147721 -0.92866446 -0.93930999
 -0.87320616 -0.91487438 -0.90202767 -0.84803664]


In [34]:
MBF_imputeScore=np.array([])

for i in range(10):
    MBF_missing=missingByFeature(X_lr_big_train,p1,p2)
    MBF_imputed=imputetMissing(MBF_missing)
    
    #doing the predective performace thing
    bestModel.fit(MBF_imputed,y_lr_big_train)
    MBF_imputeScore=np.append(MBF_imputeScore,bestModel.score(X_lr_test,y_lr_test))
    
    #do the feature selection thing
    imputed_feature_MBF=importantFeatures(bestModel,MBF_imputed,y_lr_big_train,BS_size_big)
    imputed_MBF_Index,imputed_MBF_occurrence=featureImportance(imputed_feature_MBF)
    
    print('There are',len(mostCommonFeatures(imputed_MBF_occurrence)),'important features')
    print(mostCommonFeatures(imputed_occurrence))
print(MBF_imputeScore)

There are 13 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
There are 11 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
There are 12 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
There are 15 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
There are 10 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
There are 14 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
There are 10 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
There are 11 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
There are 12 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
There are 12 important features
[101. 112. 136. 151.  47.  52.  65.  67.  90.  97.]
[ 0.17968581  0.08227616  0.09648213  0.16452436  0.29294119  0.09028483
  0.22162916  0.17156588 -0.01895995  0.26526203]
